In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/gratek/deepseek_R1_test/

/content/drive/MyDrive/gratek/deepseek_R1_test


In [3]:
from transformers import pipeline, BitsAndBytesConfig
import json
import torch
from tqdm.auto import tqdm

In [4]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [5]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"

In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    # bnb_4bit_quant_type="nf4",
    # bnb_4bit_compute_dtype=torch.bfloat16,
    # bnb_4bit_quant_storage=torch.bfloat16,
)

In [7]:
pipe = pipeline(
    "text-generation", model=model_name,
    # quantization_config=bnb_config,
    torch_dtype=torch.float16,
    # device_map="auto",
)
tokenizer = pipe.tokenizer

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/664 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/48.0k [00:00<?, ?B/s]

model-00001-of-000004.safetensors:   0%|          | 0.00/8.71G [00:00<?, ?B/s]

model-00002-of-000004.safetensors:   0%|          | 0.00/8.67G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
for lang in tqdm(["vi", "en"], desc="Language", leave=False):
    questions = json.load(open(f"questions_{lang}.json"))
    # with open(f"result_{model_name.split('/')[1]}_{lang}.txt", "w") as f:
    for type_question in tqdm(questions, desc="Type questions", leave=False):
        # print(f"Processing {type_question} questions")
        # f.write(f"Processing {type_question} questions\n")
        # f.write("="*50 + "\n")
        for question in tqdm(questions[type_question], desc="Question number", leave=False):
            # f.write(f"Question: {questions[type_question][question]}\n")
            messages = [
                {"role": "user", "content": questions[type_question][question]},
            ]
            answer = pipe(
                messages,
                max_new_tokens=1024,
                # early_stopping=True,
                do_sample=True,
                temperature=0.5,
                top_k=50,
                top_p=0.95,
                stop_strings="</think>",
                tokenizer=tokenizer
            )[0]['generated_text'][1]['content'].split("</think>")[0]
            # f.write(f"Answer:\n{answer}\n")
            # f.write("-"*50 + "\n")
            # f.flush()


Language:   0%|          | 0/2 [00:00<?, ?it/s]

Type questions:   0%|          | 0/12 [00:00<?, ?it/s]

Question number:   0%|          | 0/5 [00:00<?, ?it/s]

Question number:   0%|          | 0/5 [00:00<?, ?it/s]

Question number:   0%|          | 0/5 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Question number:   0%|          | 0/5 [00:00<?, ?it/s]